# CHAPITRE 6.1 Introduction aux traitements de séquences (textes, séries temporelles)

Dans ce chapitre on verra:

- comment pré-traiter les données textuelles pour extraire des représentations utiles

- comment utiliser les réseaux de neurones récurrents

- comment utiliser les réseaux ConvNet1D pour le traitement des séquences

les deux algorithmes fondamentaux dans l'apprentissage des séquences sont donc les réseaux de neurones récurrents (LSTM, GRU) et les ConvNet1D

Pour celà on utiliser dans les notebooks le jeux de données IMDB pour l'analyse de sentiments et des données pour faire des prédictions météo.

## I -  Traitement des données textuelles

pré-traitement très important : **vectorisation** on utilise forcement des tenseurs de nombre du coup pas de texte brute:
- segmenter en mots
- segmenter en lettre
- segmenter en n-grammes mots ou caractères :(plusieurs mots ou caractères

**tokenisation** = le fait de décomposer un texte en token, (manière, schéma utilisé pour découper le texte en tokens)

**tokens** = unité qui constituerons des variables encodé ensuite, vectorisées.

 une manière d'associé un vecteur à un token:
 
**one hot encoding** et **token embedding** (**word embedding** car souvent ce sont les mots et pas les caractères qui sont utilisés)lettre

**bag of word** ensemble de n-grams dont l'ordre n'importe pas. C'est une méthode de tokenisation où l'on considère les mots comme un ensemble et non une séquence.

les bag of words avec des n-grams est une facon de prendre en compte les mots présents ensemble mais elle est utilisée souvent uniquement en shallow learning. Cette technique reste rigide et fragile alors que les ConvNet eux peuvent facilement prendre en compte la présence de mots les uns à coté des autres (convolution). L'utilisation de n-gramm restera donc associée à une utilisation sur du shallow learning (e.g. xgboost, régression logistique, SVM...).

### I - 1 One-hot-encoding
c'est le moyen le plus simple de transformer un token en vecteur. On associe à chaque mots un indice puis on code en présence absence sa présence.

In [16]:
import numpy as np

samples = ['the cat sat on the mat','the dog ate my homework']

token_index = {}

for sample in samples:
    for word in sample.split(sep=' '):#split sur les espaces
        if word not in token_index:
            token_index[word] = len(token_index)+1

print(token_index)

{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}


In [15]:
max_length = 10

results = np.zeros(shape=(len(samples),max_length,max(token_index.values())+1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split(sep=' ')))[:max_length]:
        index = token_index.get(word)
        results[i,j,index]=1
        
results

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

On a donc une matrice pour chaque exemple qui contient en ligne la position des mots dans l'exemple et en colone la position du mots dans le dictionnaire (le vocabulaire quoi)
Maintenant on passe au codage one-hot en tant que telle:

### one hot encoding au niveau du charactère

In [36]:
import string


samples = ['je mange une tartelette a la fraise','il fait si bon dehors']

voc = string.printable

dict_char = dict(zip(range(1,len(voc)+1),voc))

max_length = 50

results = np.zeros((len(samples), #nombre d'exemple, de phrase.. d'unité statistique
                  max_length, #nombre de character pris dans l'exemple en ligne
                  max(dict_char.keys())+1)) #vocabulaire en colonne

for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = dict_char.get(character)
        results[i,j,index] = 1
        


(2, 50, 101)

Un encodage à la main peut devenir assez fastidieux s'il faut praiter le texte :
- supprimer les charactères spéciaux qui ne nous intéresse pas
- garder les mots les plus fréquents pour éviter d'avoir des matrices creuse de trop grande dimension donc des tenseurs trop grand en entrée
cependant suivant le contexte il faudra réaliser au moins le premier point à la main étant données que l'intérêt d'un charactère dépendra de la problématique suivit.

Cependant Keras dispose d'un utilitaire qui fait le travail à notre place aussi bien pour les charactères que pour les mots

### One hot encoding au niveau des séquences

In [51]:
from keras.preprocessing.text import Tokenizer # <= ici

# création d'un tokenizer qui prend uniquement les 10000 mots les plus fréquents de notre corpus
tokenizer = Tokenizer(num_words = 10000) 

#on le fit sur les textes
tokenizer.fit_on_texts(samples)

#transformes les chaine de chr en listes d'indices entiers
sequences = tokenizer.texts_to_sequences(samples)
print('une sequence d\'indice:\n',sequences)

#une manière de faire ça plus vite dans le cas d'une vectorisation binaireen ligne
one_hot_results = tokenizer.texts_to_matrix(samples,mode='binary')

one_hot_results
#ici chaque ligne est un exemple et chaque colonne est l'indice d'un mot
# ensuite codé en présence absence

#pour récupérer l'indice d'un mot
word_index = tokenizer.word_index
print('found %s unique token' %len(word_index))
word_index

une sequence d'indice:
 [[1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12]]
found 12 unique token


{'je': 1,
 'mange': 2,
 'une': 3,
 'tartelette': 4,
 'a': 5,
 'la': 6,
 'fraise': 7,
 'il': 8,
 'fait': 9,
 'si': 10,
 'bon': 11,
 'dehors': 12}

### One hot encoding avec fonction de hashage
l'intérêt est de ne pas avoir à créer un dictionnaire de mots en utilisant une fonction de hashage suffisamment grande pour ne pas avoir des vecteurs d'index qui se chevauchent


In [71]:
dimensionality = 1000
max_length = 50
results = np.zeros(shape =(len(samples), max_length,dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word))%dimensionality
        results[i,j,index]=1


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])